In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def simulate_hawkes_process_with_attributes(num_nodes=20, max_time=300, base_intensity= 1.0, alpha=0.5, beta=1.0, num_features=5,interaction_weight=0.1,noise_level=0.02,saturation_level=0.1,sat=False):
    np.random.seed(42)
    nodes = list(range(num_nodes))
    node_features = np.random.rand(num_nodes, num_features)

    # Define a function to simulate Poisson process for each edge
    def simulate_hawkes_ogata(base_intensity, alpha, beta, max_time):
        events = []
        current_time = 0
        last_event_time = -np.inf
        lambda_star = base_intensity

        while current_time < max_time:
            # Propose next event time using exponential inter-event time
            u = np.random.uniform()
            current_time += -np.log(u) / lambda_star
            
            # Calculate conditional intensity at proposed time
            conditional_intensity = base_intensity + alpha * np.sum(np.exp(-beta * (current_time - np.array(events))))

            # Accept or reject the proposed event time
            if np.random.uniform() <= conditional_intensity / lambda_star:
                events.append(current_time)
                last_event_time = current_time
                lambda_star = conditional_intensity + alpha
            else:
                lambda_star = conditional_intensity

        return events

    # # Function to simulate random walk for edge features
    # def update_edge_features(features, step_size=step_size):
    #     noise = np.random.randn(*features.shape) * step_size
    #     return features + noise

    # # Function to update node features based on interaction
    # def update_node_features(node_features, u, v, step_size=0.1):
    #     noise_u = np.random.randn(node_features.shape[1]) * step_size
    #     noise_v = np.random.randn(node_features.shape[1]) * step_size
    #     node_features[u] += noise_u
    #     node_features[v] += noise_v
    #     return node_features
    
    def update_node_features(node_features, u, v, interaction_weight=interaction_weight):
        # Simulate feature updates based on interaction between u and v
        node_features[u] += interaction_weight * (node_features[v] - node_features[u]) + np.random.randn(num_features) * 0.02
        node_features[v] += interaction_weight * (node_features[u] - node_features[v]) + np.random.randn(num_features) * 0.02
        return node_features

    def update_node_features_with_sat(node_features, u, v, interaction_weight=interaction_weight, noise_level=noise_level, saturation_level=saturation_level):
        # Simulate feature updates based on interaction between u and v
        interaction_effect = interaction_weight * (node_features[v] - node_features[u])
        noise_u = np.random.randn(num_features) * noise_level
        noise_v = np.random.randn(num_features) * noise_level
        
        node_features[u] += np.clip(interaction_effect + noise_u, -saturation_level, saturation_level)
        node_features[v] += np.clip(-interaction_effect + noise_v, -saturation_level, saturation_level)
        
        return node_features

    initial_events = []

    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if i != j:
                times = simulate_hawkes_ogata(base_intensity, alpha, beta, max_time)
                for t in times:
                    initial_events.append((i, j, t))

    initial_events.sort(key=lambda x: x[2])
    adj_matrix = np.zeros((num_nodes, num_nodes))
    final_events = []

    for event in initial_events:
        u, v, time = event

        # Update node features dynamically
        if sat == True:
            node_features = update_node_features_with_sat(node_features, u, v, interaction_weight=interaction_weight, noise_level=noise_level, saturation_level=saturation_level)
        else:
            node_features = update_node_features(node_features, u, v, interaction_weight=interaction_weight)

        sim = cosine_similarity([node_features[u]], [node_features[v]])[0][0]
        
        if adj_matrix[u, v] == 0 and np.random.rand() < prob:
            adj_matrix[u, v] = 1
            adj_matrix[v, u] = 1
            final_events.append((u, v, time, 0, sim))  # Type 0 for first activation
        if adj_matrix[u, v] == 1:
            final_events.append((u, v, time, 1, sim))  # Type 1 for subsequent activations

    events_df = pd.DataFrame(final_events, columns=['u', 'v', 'time', 'k', 'similarity'])

    # Output the final events with dynamic edge features
    events_df.to_csv('/simulated_data/final_hawkes_with_features.csv', index=False)
    print("Final events with dynamic edge features have been generated and saved to 'final_hawkes_with_features.csv'.")


In [ ]:
simulate_hawkes_process_with_attributes(num_nodes=20, max_time=300, base_intensity= 1, alpha=0.5, beta=1.0, num_features=5,interaction_weight=0.01,sat=True)  